In [1]:
import symforce
symforce.set_epsilon_to_symbol()
import cv2
import numpy as np
import time
import symforce.symbolic as sf
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer
from symforce.values import Values

In [2]:
def pixel2cam(p, K):
    return np.array([(p[0] - K[0, 2]) / K[0, 0],
                     (p[1] - K[1, 2]) / K[1, 1]])

In [3]:
def findFeatureMatches(img_1, img_2):

    # Keypoints and their descriptors
    keypoints_1, keypoints_2 = [], []
    descriptors_1, descriptors_2 = [], []

    orb = cv2.ORB_create()

    keypoints_1, descriptors_1 = orb.detectAndCompute(img_1, None)
    keypoints_2, descriptors_2 = orb.detectAndCompute(img_2, None)

    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

    matches = matcher.match(descriptors_1, descriptors_2)

    # Sort and remove outliers
    matches = sorted(matches, key = lambda x: x.distance)

    min_dist = matches[0].distance
    max_dist = matches[len(matches) - 1].distance

    print(f"Min dist: {min_dist}, Max dist: {max_dist}")

    good_matches = []

    for i in range(descriptors_1.shape[0]):
        if matches[i].distance <= max(2 * min_dist, 30.0):
            good_matches.append(matches[i])

    return keypoints_1, keypoints_2, good_matches

In [4]:
# load images
img_1 = cv2.imread("./1.png")
img_2 = cv2.imread("./2.png")
keypoints_1, keypoints_2, matches = findFeatureMatches(img_1, img_2)
print(f"Number of matches: {len(matches)}")

d1 = cv2.imread("./1_depth.png", cv2.IMREAD_UNCHANGED)
K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])

Min dist: 4.0, Max dist: 94.0
Number of matches: 79


In [6]:
# these have to be Nx3 and Nx2 numpy arrays
pts_3d = np.empty((0, 3))
pts_2d = np.empty((0, 2))
for m in matches:
    d = d1[int(keypoints_1[m.queryIdx].pt[1]),
           int(keypoints_1[m.queryIdx].pt[0])]
    if d == 0:
        continue
    dd = d/5000.0
    p1 = pixel2cam(keypoints_1[m.queryIdx].pt, K)
    pts_3d = np.vstack([pts_3d, np.array([p1[0]*dd, p1[1]*dd, dd])])
    pts_2d = np.vstack([pts_2d, keypoints_2[m.trainIdx].pt])
print(f"3d-2d pairs: {len(pts_3d)}")

3d-2d pairs: 75


In [7]:
t1 = time.time()
temp, r, t = cv2.solvePnP(pts_3d, pts_2d, K, np.array([]))
R = cv2.Rodrigues(r)[0]
t2 = time.time()
print(f"Solved pnp in opencv time: {t2-t1}")
print(f"R = {R}")
print(f"t = {t}")

Solved pnp in opencv time: 0.4151146411895752
R = [[ 0.99790591 -0.0509194   0.03988747]
 [ 0.04981866  0.99836232  0.02812094]
 [-0.04125405 -0.02607491  0.99880839]]
t = [[-0.12678213]
 [-0.00843949]
 [ 0.06034935]]


In [8]:
def compute_error(T: sf.Pose3, 
                 measurement: sf.V2,
                 pos: sf.V3,
                 K: sf.M33) -> sf.V2:
    pos_pixel = K*(T*pos) 
    pos_pixel = pos_pixel/pos_pixel[2]
    return measurement - sf.V2(pos_pixel[0], pos_pixel[1])

In [9]:
# Redo K as a Symforce type
K = sf.M33([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])

initial_values = Values(
    T = sf.Pose3(),
    measurements = [sf.V2(x[0], x[1]) for x in pts_2d.tolist()],
    points_3d = [sf.V3(x[0], x[1], x[2]) for x in pts_3d.tolist()],
    K = K,
    epsilon = sf.numeric_epsilon,
)


factors = []
for i in range(len(pts_2d)):
    factors.append(Factor(
        residual=compute_error,
        keys=["T", f"measurements[{i}]", f"points_3d[{i}]", "K"],
    ))

In [10]:
optimizer = Optimizer(
    factors=factors,
    optimized_keys=["T"],
    debug_stats=True,
    params=Optimizer.Params(iterations=1000)
)
t1 = time.time()
result = optimizer.optimize(initial_values)
t2 = time.time()
print(f"Symforce solved in {t2-t1} seconds")
pose = result.optimized_values.get('T').to_homogenous_matrix()
display(pose)

[2023-05-19 10:13:25.664] [info] LM<sym::Optimize> [iter    0] lambda: 1.000e+00, error prev/linear/new: 20258.878/0.000/164.198, rel reduction: 0.99190
[2023-05-19 10:13:25.673] [info] LM<sym::Optimize> [iter    1] lambda: 2.500e-01, error prev/linear/new: 164.198/0.000/149.882, rel reduction: 0.08719
Symforce solved in 0.10136127471923828 seconds
[2023-05-19 10:13:25.679] [info] LM<sym::Optimize> [iter    2] lambda: 6.250e-02, error prev/linear/new: 149.882/0.000/149.882, rel reduction: 0.00000


array([[ 0.99790591, -0.05091941,  0.03988745, -0.1267821 ],
       [ 0.04981867,  0.99836232,  0.02812093, -0.00843948],
       [-0.04125403, -0.02607491,  0.99880839,  0.06034935],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])